In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=3df29ede1f35853feb5aa59b74498ec88c5387b03dda351999f28f2b7d981257
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 2081.2008060397466 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_10_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=905357, Mon Apr 13 18:39:44 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.776979431742557e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.0
    2     30 2.857334739023157e+07 1.0e+00 2.30e+00  2e+00  2e+00 0:00.0
    3     45 2.524140784626435e+07 1.1e+00 2.24e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [42003633.15279736] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [46762409.15108949] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -3.935496038326759e+07 1.4e+00 4.25e-01  4e-01  4e-01 0:01.8
  200   3000 -4.335781938685180e+07 1.5e+00 2.91e-02  3e-02  3e-02 0:03.5
  300   4500 -4.337446416296532e+07 1.6e+00 1.76e-03  2e-03  2e-03 0:05.2
  400   6000 -4.337452947717173e+07 1.6e+00 1.34e-04  1e-04  1e-04 0:07.0
  500   7500 -4.337452985848365e+07 1.7e+00 9.28e-06  7e-06  8e-06 0:08.7
  600   9000 -4.337452986005213e+07 1.8e+00 7.14e-07  6e-07  6e-07 0:10.4
  700  10500 -4.337452986006159e+07 1.9e+00 5.46e-08  4e-08  5e-08 0:12.1
  800  12000 -4.337452986006167e+07 2.0e+00 3.06e-08  2e-08  3e-08 0:13.8
  900  13500 -4.337452986006164e+07 2.2e+00 1.83e-08  1e-08  2e-08 0:15.5
 1000  15000 -4.337452986006165e+07 2.4e+00 1.43e-08  1e-08  1e-08 0:17.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -4.337452986006168e+07 2.6e+00 1.04e-08  7e-09  9e-09 0:19.0
 1200  18000 -4.337452986006165e+07 2.7e+00 6.91e-09  5e-09  6e-09 0:20.8
 1300  19500 -4.337452986006165e+07 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [39119013.58049539] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39119013.580495, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39119013.580495, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -8.281039191241513e+07 1.5e+00 4.61e-01  4e-01  5e-01 0:02.1
  200   3000 -9.351145943162166e+07 1.9e+00 1.14e-01  1e-01  1e-01 0:04.2
  300   4500 -9.617992599362776e+07 2.9e+00 6.39e-02  5e-02  8e-02 0:06.4
  400   6000 -9.665755165671961e+07 3.4e+00 3.01e-02  2e-02  5e-02 0:08.6
  500   7500 -9.686729684866099e+07 5.1e+00 4.13e-02  3e-02  9e-02 0:10.6
  600   9000 -9.705084659420194e+07 8.5e+00 2.16e-02  1e-02  8e-02 0:12.7
  700  10500 -9.707046783356473e+07 9.0e+00 4.22e-03  2e-03  2e-02 0:14.7
  800  12000 -9.707186034480986e+07 8.9e+00 1.26e-03  6e-04  5e-03 0:16.8
  900  13500 -9.707190279568726e+07 9.2e+00 1.90e-04  9e-05  7e-04 0:19.0
 1000  15000 -9.707190367711875e+07 9.1e+00 2.32e-05  1e-05  8e-05 0:21.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -9.707190369032079e+07 9.3e+00 2.71e-06  1e-06  9e-06 0:23.3
 1200  18000 -9.707190369062309e+07 9.3e+00 6.72e-07  3e-07  2e-06 0:25.5
 1300  19500 -9.707190369063580e+07 9.4e

...

  100   1500 -8.164771845286450e+07 1.6e+00 6.57e-01  6e-01  7e-01 0:02.1
  200   3000 -9.724438180000372e+07 1.9e+00 1.91e-01  2e-01  2e-01 0:04.2
  300   4500 -1.005175145533054e+08 2.7e+00 9.30e-02  7e-02  1e-01 0:06.3
  400   6000 -1.014769076593169e+08 3.2e+00 6.69e-02  5e-02  9e-02 0:08.5
  500   7500 -1.020236085221816e+08 4.8e+00 3.56e-02  2e-02  6e-02 0:10.6
  600   9000 -1.022337998327015e+08 6.9e+00 2.11e-02  1e-02  5e-02 0:12.8
  700  10500 -1.022775939601517e+08 7.2e+00 1.40e-02  8e-03  3e-02 0:15.0
  800  12000 -1.022877614917798e+08 7.3e+00 2.61e-03  1e-03  6e-03 0:17.1
  900  13500 -1.022881035282504e+08 7.4e+00 7.09e-04  4e-04  2e-03 0:19.2
 1000  15000 -1.022881532703970e+08 8.3e+00 2.07e-04  1e-04  5e-04 0:21.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.022881544722435e+08 8.5e+00 2.88e-05  1e-05  6e-05 0:23.5
 1200  18000 -1.022881545028507e+08 8.5e+00 6.35e-06  3e-06  1e-05 0:25.7
 1300  19500 -1.022881545037634e+08 8.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [37255074.53322896] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [38235453.30693674] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -3.979405413417404e+07 1.4e+00 3.64e-01  3e-01  4e-01 0:02.1
  200   3000 -4.335921629325310e+07 1.5e+00 2.58e-02  2e-02  3e-02 0:04.2
  300   4500 -4.337445279547685e+07 1.6e+00 1.84e-03  2e-03  2e-03 0:06.2
  400   6000 -4.337452953377680e+07 1.6e+00 1.16e-04  1e-04  1e-04 0:08.3
  500   7500 -4.337452985841856e+07 1.7e+00 1.03e-05  8e-06  9e-06 0:10.4
  600   9000 -4.337452986005587e+07 1.8e+00 5.63e-07  4e-07  5e-07 0:12.4
  700  10500 -4.337452986006158e+07 1.8e+00 5.77e-08  4e-08  5e-08 0:14.5
  800  12000 -4.337452986006162e+07 2.1e+00 2.21e-08  2e-08  2e-08 0:16.5
  900  13500 -4.337452986006162e+07 2.3e+00 2.48e-08  2e-08  2e-08 0:18.6
 1000  15000 -4.337452986006162e+07 2.4e+00 1.46e-08  1e-08  1e-08 0:20.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -4.337452986006162e+07 2.6e+00 1.35e-08  9e-09  1e-08 0:22.7
 1200  18000 -4.337452986006162e+07 2.8e+00 1.12e-08  7e-09  9e-09 0:24.7
 1300  19500 -4.337452986006159e+07 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [39119013.58049539] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39119013.580495, sigma=2.39e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=39119013.580495, sigma=2.39e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 -8.368732574381717e+07 1.5e+00 6.65e-01  6e-01  7e-01 0:02.2
  200   3000 -9.937347234676124e+07 2.1e+00 2.14e-01  2e-01  2e-01 0:04.3
  300   4500 -1.036580952079938e+08 2.6e+00 9.63e-02  7e-02  1e-01 0:06.5
  400   6000 -1.057533908495485e+08 3.5e+00 5.36e-02  4e-02  8e-02 0:08.6
  500   7500 -1.060460637502597e+08 3.7e+00 1.73e-02  1e-02  2e-02 0:10.8
  600   9000 -1.060806283606766e+08 4.4e+00 1.21e-02  7e-03  2e-02 0:12.9
  700  10500 -1.061272906392176e+08 7.4e+00 1.40e-02  8e-03  4e-02 0:15.1
  800  12000 -1.061367689752972e+08 7.9e+00 4.56e-03  2e-03  1e-02 0:17.3
  900  13500 -1.061382947619952e+08 8.1e+00 2.12e-03  1e-03  6e-03 0:19.4
 1000  15000 -1.061385127343266e+08 8.3e+00 6.47e-04  3e-04  2e-03 0:21.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.061385268972439e+08 8.6e+00 1.51e-04  6e-05  4e-04 0:23.7
 1200  18000 -1.061385279713401e+08 8.7e+00 5.53e-05  2e-05  1e-04 0:25.8
 1300  19500 -1.061385282113107e+08 9.9e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F10_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:50])
    SVM_Fun [i] = F10(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F10(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([9957811.44385473]), count=array([1])) 0.12706570253021507
SVM
ModeResult(mode=array([10616304.30605206]), count=array([1])) 31796532.93263333
ELN
ModeResult(mode=array([6259100.83079547]), count=array([1])) 4432000.043399737
